<a href="https://colab.research.google.com/github/IzzyBrand/robotBottleFlip/blob/main/traj_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import ipywidgets
import pydot
import os
from IPython.display import display, SVG

import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20201120', drake_build='nightly')

if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import pydrake.all
from manipulation.scenarios import AddIiwa, AddWsg, AddRgbdSensors, AddShape
from manipulation.utils import FindResource

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, TrajectorySource, JacobianWrtVariable)

from pydrake.all import (
    DiagramBuilder,
    RenderEngineVtkParams, MakeRenderEngineVtk, DepthCameraProperties, RgbdSensor, RigidTransform, RollPitchYaw,
    ColorRenderCamera, DepthRenderCamera, CameraInfo, ClippingRange, RenderCameraCore, DepthRange
)
from pydrake.symbolic import Expression
    
from pydrake.all import RigidTransform, RotationMatrix, SpatialVelocity, PiecewisePolynomial

from pydrake.all import eq, MathematicalProgram, Solve

from pydrake.all import RenderEngineVtkParams, DepthCameraProperties, ModelInstanceIndex, DepthImageToPointCloud, BaseField, LeafSystem, AbstractValue

from google.colab.patches import cv2_imshow

In [ ]:
import requests
resp = requests.get("https://raw.githubusercontent.com/IzzyBrand/robotBottleFlip/main/models/bottle.sdf")
if resp.status_code == 200:
    bottle_sdf = resp.text

# Optimize using forward kinematics

In [ ]:
def optimize_arm_trajectory(X_final, # bottle release pose (RigidTransform)
                            V_final, # bottle release velocity (np.ndarray [6])
                            q_init,  # iiwa initial pose (np.ndarray [7])
                            X_GB,    # pose of the bottle in the gripper frame (RigidTransform)
                            T=50):   # number of timestep

    # set up a plant for use in inverse kinematics
    plant = pydrake.multibody.plant.MultibodyPlant(0.0)
    iiwa = AddIiwa(plant)
    plant.Finalize()
    plant = plant.ToSymbolic() # suggested in piazza post @391
    context = plant.CreateDefaultContext()
    L7 = plant.GetBodyByName('iiwa_link_7') # get the last link for later use

    # transform from link 7 to the WSG
    X_L7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
    # and from link 7 to the bottle. Note that this needs to be an Expression
    # to use in the symbolic computations
    X_L7B = X_L7G.multiply(X_GB)

    # set up the optimization problem
    prog = MathematicalProgram()

    # with optimization variables
    q = prog.NewContinuousVariables(T, 7, 'q')     # joint position
    qd = prog.NewContinuousVariables(T, 7, 'qd')   # joint velocity
    # qdd = prog.NewContinuousVariables(T, 7, 'qdd') # joint acceleration
    h = prog.NewContinuousVariables(1, 'h')        # timestep

    # put the plant at the final state
    plant.SetPositions(context, q[-1])
    plant.SetVelocities(context, qd[-1])
    # compute the resulting bottle pose and velocity
    X_WL7 = plant.EvalBodyPoseInWorld(context , L7) # pose of link 7 (symbolic)
    X_WB = X_WL7.multiply(X_L7B.cast[Expression]()) # pose of the bottle at the final timestep

    print('hello')
    # compute the spatial velocity jacobian of the bottle
    J_B = plant.CalcJacobianSpatialVelocity(context,
                                            JacobianWrtVariable.kQDot, # this often is .kv
                                            L7.body_frame(),
                                            X_L7B.translation(),
                                            plant.world_frame(),
                                            plant.world_frame())
    print('here')
    # and finally, the spatial velocity of the bottle at the last timestep
    V_WB = J_B.dot(qd[-1]) # [roll, pitch, yaw, x, y, z]
    
    # add trajectory constraints
    # NOTE(izzy): for some reason I can't make these AddLinearConstraint...
    print('before timestep constraints')
    for t in range(T-1):
        prog.AddConstraint(eq(q[t+1], q[t] + h*qd[t+1]))    # position
        # prog.AddConstraint(eq(qd[t+1], qd[t] + h*qdd[t])) # velocity
        # prog.AddBoundingBoxConstraint([-0.1]*7, [0.1]*7, (qd[t+1] - qd[t])/h) # bound the joint accelerations
        print(t)
    # prog.AddConstraint(eq(q[1:], q[:-1] + h*qd[:-1]))    # position
    # prog.AddConstraint(eq(qd[1:], qd[:-1] + h*qdd[:-1])) # velocity
    prog.AddConstraint(eq(q[0], q_init)) # initial joint position

    # add timestep constraint
    prog.AddBoundingBoxConstraint(0.001, 0.02, h)

    # add final position constraint
    print('adding pose constraints')
    for i in range(3):
        prog.AddConstraint(eq(X_WB.rotation().matrix()[i], X_final.rotation().matrix()[i]))
        print(i)

    
    print('adding velocity constraint')
    # add final velocity constraint
    prog.AddConstraint(eq(V_WB, V_final))
    print('done')

    # add bottle acceleration constraints
    # TODO

    # add joint acceleration costs
    # TODO

    # result = Solve(prog)
    # if result.is_success():
    #     return (result.GetSolution(var) for var in (q,qd,qdd))
    # else:
    #     print('Optimization Failed')
    return prog

prog = optimize_arm_trajectory(X_final = RigidTransform(),
                        V_final = np.zeros(6, dtype=np.float64),
                        q_init = np.zeros(7),
                        X_GB = RigidTransform())

#Optimize using inverse kinematics